In [75]:
import camelot
import pandas as pd

pdf_path = "MasterCard Statement-9072 2025-12-08-1.pdf"
tables = camelot.read_pdf(
    pdf_path,
    flavor="stream"   # RBC statements work best with stream
)
df_raw = pd.concat([t.df for t in tables], ignore_index=True)

print(df_raw.shape)
print(df_raw.head(20))
print(df_raw.tail(20))


(150, 5)
              0        1                                   2           3  \
0                                                                          
1                                      CASSANDRA LITWINOWICH               
2                              5415 90** **** 9072 - PRIMARY               
3   TRANSACTION  POSTING                                                   
4                                       ACTIVITY DESCRIPTION  AMOUNT ($)   
5          DATE     DATE                                                   
6                                                                          
7        NOV 12   NOV 13               UBER* EATS TORONTO ON      $38.09   
8                                                                          
9                                    82300095316500147453941               
10       NOV 12   NOV 13     UBER CANADA/UBEREATS TORONTO ON       $4.00   
11                                   55134425316800140713375               
12 

In [76]:
df = df_raw.copy()
df.columns = ["transaction_date", "posting_date", "description", "amount", "extra"]
df = df.fillna("")

    

In [77]:
import re

mask = (
    df["transaction_date"].str.match(
        r"(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)\s+\d{1,2}",
        na=False
    )
    & df["amount"].str.contains(r"\$", na=False)
)

df_tx = df[mask].copy()


In [78]:
df_tx["amount"] = (
    df_tx["amount"]
    .str.replace("$", "", regex=False)
    .str.replace(",", "", regex=False)
    .astype(float)
)


In [79]:
print(df_tx.head())


   transaction_date posting_date                         description  amount  \
7            NOV 12       NOV 13               UBER* EATS TORONTO ON   38.09   
10           NOV 12       NOV 13     UBER CANADA/UBEREATS TORONTO ON    4.00   
13           NOV 14       NOV 17  PUBLIC MOBILE SELF-SER EDMONTON AB   20.95   
16           NOV 18       NOV 19     UBER CANADA/UBEREATS TORONTO ON   40.32   
19           NOV 18       NOV 19     UBER CANADA/UBEREATS TORONTO ON    5.87   

                                               extra  
7                                                     
10  Customer Service / Lost & Stolen\n1-800-769-2512  
13                                                    
16                         PAYMENTS & INTEREST RATES  
19                                                    


In [80]:
df_tx.to_csv("rbc_mastercard_transactions.csv", index=False)
